### Part1: Preprocessing Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
## checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/project_codes/backend/data_engineering/data_modelling_casandra


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

### Part2: Cassandra Insert

**Cassandra Cluster**

In [4]:
#### run cassandra -f in the terminal to start cassandra service ####
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
# To establish connection and begin executing queries, need a session
session = cluster.connect()

**Keyspace Creation**

In [5]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS songdb 
WITH REPLICATION = 
{'class' : 'SimpleStrategy', 'replication_factor' : 1}
""")

**Set Keyspace**

In [6]:
session.set_keyspace("songdb")

**Table Creation Based on Queries**

**1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

For table session_data; session_id and iteminsession is used a primary key, as we are filterig on sessionid and iteminsessionid

In [7]:
table1_create="""
CREATE TABLE IF NOT EXISTS session_data 
(
sessionid int,
iteminsession int,
artist text,
song text,
length float,
PRIMARY KEY (sessionid,iteminsession)
)
"""
session.execute(table1_create)

In [8]:
table1_insert = """
INSERT INTO songdb.session_data (sessionid,iteminsession,artist,song,length) VALUES (%s,%s,%s,%s,%s);
"""

In [9]:
import csv
with open("event_datafile_new.csv","r",encoding="utf8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        session.execute(table1_insert,(
                                      int(row.get('sessionId')),
                                      int(row.get('itemInSession')),
                                      row.get('artist'),
                                      row.get('song'),
                                      float(row.get('length'))))

In [10]:
table1_select = """
select artist,song,length from songdb.session_data
where sessionid=338 and iteminsession=4
"""

In [11]:
results = session.execute(table1_select)

In [12]:
[r for r in results]

[Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)]

**2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

For table artist_user_data; userid and sessionid are used as composite key and iteminsession id is used as clustering key as data needs to be sorted by item in session

In [13]:
table2_create="""
CREATE TABLE IF NOT EXISTS artist_user_data
(
userid int,
sessionid int,
iteminsession int,
artist text,
song text,
firstname text,
lastname text,
PRIMARY KEY ((userid,sessionid),iteminsession)
)
"""
session.execute(table2_create)

In [14]:
table2_insert = """
INSERT INTO songdb.artist_user_data (userid,sessionid,iteminsession,artist,song,firstname,lastname
            ) VALUES (%s,%s,%s,%s,%s,%s,%s);
"""

In [15]:
with open("event_datafile_new.csv","r",encoding="utf8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        session.execute(table2_insert,(
                                      int(row.get('userId')),
                                      int(row.get('sessionId')),
                                      int(row.get('itemInSession')),
                                      row.get('artist'),
                                      row.get('song'),
                                      row.get('firstName'),
                                      row.get('lastName')
                                      ))

In [16]:
table2_select = """
select artist,song,firstname,lastname from songdb.artist_user_data
where userid=10 and sessionid=182
"""

In [17]:
results = session.execute(table2_select)

In [18]:
[r for r in results]

[Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz'),
 Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')]

**3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

For table searchbysong; song is used as partition key and userid is used as clustering key

In [19]:
table3_create="""
CREATE TABLE IF NOT EXISTS searchbysong
(
song text,
userid int,
firstname text,
lastname text,
PRIMARY KEY ((song),userid)
)
"""
session.execute(table3_create)

In [20]:
table3_insert = """
INSERT INTO songdb.searchbysong (song,userid,firstname,lastname) VALUES (%s,%s,%s,%s);
"""

In [21]:
with open("event_datafile_new.csv","r",encoding="utf8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        session.execute(table3_insert,(
                                      row.get('song'),
                                      int(row.get('userId')),
                                      row.get('firstName'),
                                      row.get('lastName')
                                      ))

In [22]:
table3_select = """
select firstname,lastname from songdb.searchbysong
where song='All Hands Against His Own'
"""

In [23]:
results = session.execute(table3_select)

In [24]:
[r for r in results]

[Row(firstname='Jacqueline', lastname='Lynch'),
 Row(firstname='Tegan', lastname='Levine'),
 Row(firstname='Sara', lastname='Johnson')]

**Drop Tables**

In [25]:
drop_sessiondata="""
DROP TABLE IF EXISTS session_data
"""
drop_artistuserdata = """
DROP TABLE IF EXISTS artist_user_data
"""
drop_searchbysong = """
DROP TABLE IF EXISTS search_bysong
"""

In [26]:
session.execute(drop_sessiondata)

In [27]:
session.execute(drop_artistuserdata)

In [28]:
session.execute(drop_searchbysong)